## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [1]:
import pandas as pd
df = pd.read_csv('chronic_kidney_disease_full.csv')

In [2]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

Answer: 
- Check for missing values
- Check whether there would be class imbalance. ckd = 250 (62.5%) and notckd = 150 (37.5%) is still acceptable.
- Check for correlation between each pair of features. This is very important as we will be using regression later.

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [3]:
df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

In [4]:
# Show percentage instead
(df.isnull().sum() / len(df)) * 100

age       2.25
bp        3.00
sg       11.75
al       11.50
su       12.25
rbc      38.00
pc       16.25
pcc       1.00
ba        1.00
bgr      11.00
bu        4.75
sc        4.25
sod      21.75
pot      22.00
hemo     13.00
pcv      17.75
wbcc     26.50
rbcc     32.75
htn       0.50
dm        0.50
cad       0.50
appet     0.25
pe        0.25
ane       0.25
class     0.00
dtype: float64

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

In [5]:
len(df) - df.dropna(axis = 0, how = 'any', inplace = False).shape[0]

242

In [6]:
(len(df) - df.dropna(axis = 0, how = 'any', inplace = False).shape[0])/len(df) * 100

60.5

Answer: We are dropping 242 rows of data. This is 60% of the data. The downside is that we will have much lesser data to work with. For some rows, there are many non missing values for other columns as well. Hence, we are losing a lot of important information.

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

Answer: As the data is most likely entererd into the system manually by humans, there is a chance that typo errors might occur and this will lead to inaccuracies.

Certain medical equipment that measure specific health metrics might not be calibrated properly.

Different people have different standards for subjective measures like appetite. What I think is good might be what others think is poor, unless there is a qualitative guideline to determine poor or good.

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Answer: Tell everyone that they have CKD (which is definitely not ideal as this will cause unnecessary panic)

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: We minimise false negative and maximise recall.

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

Answer: This will lead to social panic and anxiety. As everyone who went for the test will be diagnosed with CKD, all of them will opt for medical treatment and medicine, which will greatly inflate the prices of such medical commodities.

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

Answer: Tell everyone they do not have CKD.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: Minimise false positive and maximise precision.

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

Answer: People who are positives will die while thinking that they are perfectly healthy.

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [7]:
df['class'] = [1 if i == 'ckd' else 0 for i in df['class']]

In [8]:
df['class'].value_counts()

1    250
0    150
Name: class, dtype: int64

In [9]:
# Let's use all features first
df_dummies = pd.get_dummies(df, drop_first = True)

In [10]:
df_dummies.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo',
       'pcv', 'wbcc', 'rbcc', 'class', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_poor', 'pe_yes', 'ane_yes'],
      dtype='object')

In [11]:
X = df_dummies.drop('class', axis=1)
y = df_dummies['class']
# Impute missing values with median for simplicity
X_imp = X.fillna(X.median())

In [12]:
from sklearn.model_selection import train_test_split
# Use stratify = y to ensure that the proportion of classes is retained even after splitting
X_train, X_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.3, random_state=42, stratify=y)

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

C:\Users\Razer\anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [15]:
logreg.score(X_train, y_train)

0.9321428571428572

In [16]:
logreg.score(X_test, y_test)
# Severe overfitting

0.8666666666666667

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [17]:
import numpy as np
np.exp(logreg.coef_[0])

array([0.95006116, 1.12493255, 1.00229895, 1.14095099, 1.01436931,
       1.0543119 , 1.03754357, 1.17688743, 1.10049294, 0.99128044,
       0.75701225, 0.55374159, 1.00017406, 0.94294307, 0.90642774,
       0.96559109, 1.01126179, 1.00390147, 1.04204813, 1.03708156,
       1.00659878, 1.02648163, 1.03082388, 1.01225188])

In [18]:
X_imp.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo',
       'pcv', 'wbcc', 'rbcc', 'rbc_normal', 'pc_normal', 'pcc_present',
       'ba_present', 'htn_yes', 'dm_yes', 'cad_yes', 'appet_poor', 'pe_yes',
       'ane_yes'],
      dtype='object')

As age increases by 1 unit (in year), a person is 0.95 times as likely to contract CKD when all other predictors are constant.

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

A person with a poor appetite is 1.02 times as likely to contract CKD when all other predicotrs are held constant.

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

Answer: Results are easy to interpret and it is easy to implement.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

It makes sense to generate a confusion matrix as there are 2 classes and we can calculate the precision, recall and f scores.

In [19]:
y_pred = logreg.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.82      0.82        45
           1       0.89      0.89      0.89        75

    accuracy                           0.87       120
   macro avg       0.86      0.86      0.86       120
weighted avg       0.87      0.87      0.87       120



In [20]:
confusion_matrix(y_test, y_pred)

array([[37,  8],
       [ 8, 67]], dtype=int64)

### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

Answer: In my opinion, it will be better to maximise recall. This means that we are minimising the number of false negatives and playing it safe by telling more people that they have CKD as opposed to telling people who have CKD that they are healthy. Personally I feel that this is more ideal as I would rather tell someone healthy that he/she has CKD instead of telling someone with CKD that he/she has no CKD.

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

Answer: It is difficult for the machine learning model to learn about characteristics of minority classes when there is insufficient data.

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

Answer: If there is very little data on people who contract rare diseases, there is insufficient information and it becomes much more challenging for doctors to identify root causes and propose treatments. This is the same thing for machine learning models as well.

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [21]:
df_sample = df[df['class'] == 1].sample(200000, replace=True, random_state=42)
len(df_sample)

200000

In [22]:
df2 = pd.concat([df, df_sample])
df2.shape

(200400, 25)

In [23]:
df2.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,1
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,1
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,1
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,1
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,1


In [24]:
df2['class'].value_counts()

1    200250
0       150
Name: class, dtype: int64

In [25]:
# Get percentage instead
df2['class'].value_counts(normalize=True) * 100

1    99.92515
0     0.07485
Name: class, dtype: float64

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

In [26]:
df2_dummies = pd.get_dummies(df2, drop_first = True)
X2 = df2_dummies.drop('class', axis=1)
y2 = df2_dummies['class']
X2_imp = X2.fillna(X2.median())
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_imp, y2, test_size=0.3, random_state=42)
logreg2 = LogisticRegression()
logreg2.fit(X2_train, y2_train)

C:\Users\Razer\anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [27]:
logreg.score(X_train, y_train)

0.9321428571428572

In [28]:
logreg2.score(X2_train, y2_train)

0.9993370402053037

In [29]:
logreg.score(X_test, y_test)

0.8666666666666667

In [30]:
logreg2.score(X2_test, y2_test)

0.9991017964071857

In [31]:
y2_pred = logreg2.predict(X2_test)
print(classification_report(y2_test, y2_pred))

              precision    recall  f1-score   support

           0       1.00      0.04      0.07        56
           1       1.00      1.00      1.00     60064

    accuracy                           1.00     60120
   macro avg       1.00      0.52      0.53     60120
weighted avg       1.00      1.00      1.00     60120



Model score and precision are very high due to the overwhelming % of majority classes. This is an issue because the model most likely would not be able to predict the minority class well (i.e. model is not good at diagnosing people without CKD accurately). The model si most likely biased.

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!